In [13]:
import itertools
import math
from enum import Enum

import numpy as np
import drawSvg as draw

In [14]:
stripwidth = 10

def render(strip, stripwidth):
    striplength = 0
    
    for element in strip:
        striplength += element.get_length()
    
    d = draw.Drawing(striplength,stripwidth)
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('strip.svg')
    d.rasterize()
    return d

In [15]:
class Plain:
    def __init__(self, length):
        self.length = length
    def get_length(self,stripwidth=10):
        return self.length
    
    
def add_plain(l,strip = []):
    strip.append(Plain(l))
    
strip = []
add_plain(100,strip)
render(strip, stripwidth)

In [16]:
class Fold:
    def __init__(self, mv, angle):
        self.mv = mv
        self.angle = angle
    def get_length(self,stripwidth):
        return stripwidth*math.tan(math.radians(90-self.angle))
class MV(Enum):
    MOUNTAIN=0
    VALLEY=1 
def add_fold(mv,angle,strip = []):
    strip.append(Fold(mv,angle))
def render (strip, stripwidth):
    striplength=0
    for element in strip: striplength += abs(element.get_length(stripwidth))   
    d = draw.Drawing(striplength,stripwidth)
    x = 0
    for element in strip:
        if type(element) is Plain:
            x +=element.length
        if type(element) is Fold:
            l = element.get_length(stripwidth)    
            x1,x2 = x, x+abs(l) 
            if l <0: x1,x2 = x2,x1
            d.append(draw.Line(x1,0,x2,stripwidth,stroke='red' if element.mv is MV.MOUNTAIN else 'blue', fill='none')) 
            x+= abs(l)
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('strip.svg')
    d.rasterize()
    return d

add_fold(MV.MOUNTAIN, -45, strip)
add_fold(MV.MOUNTAIN, 45, strip)
add_plain(100, strip)
add_fold(MV.VALLEY, -30,strip)

render(strip, stripwidth)

In [7]:
def add_corner_gadget (mv, strip=[]):
    add_fold(mv,45,strip)
    add_fold(mv,-45,strip)
add_corner_gadget(MV.MOUNTAIN, strip)
render (strip, stripwidth)

In [17]:
class Direction(Enum):
    L=-1
    R=1
    

def add_corner_gadget (mv, d, strip=[]):
    add_fold(mv,45*d.value,strip)
    add_fold(mv,-45*d.value,strip)

add_corner_gadget(MV.MOUNTAIN,Direction.R,strip)
add_plain(100,strip)
add_corner_gadget(MV.MOUNTAIN,Direction.L, strip)
render (strip,stripwidth)

In [18]:
# returns the distance between two points
def get_distance(a,b): return np.linalg.norm(np.array(a) - np.array(b))

# returns the angle given these sides
def cosinelaw(adj_side1,adj_side2,opp_side): return math.acos((adj_side1 ** 2 + adj_side2 ** 2 - opp_side **2) / (2*adj_side1 * adj_side2))

# returns the length of the over-extended strip
def corner_extension(gamma,beta,direction,strip_width): return math.tan(0.5*math.pi - (gamma if direction is Direction.L else beta))*strip_width

# calculates all angles given a triangle with vertices a,b,c
def calculate_angles(a, b, c):
    ab = get_distance(a,b)
    bc = get_distance(b,c)
    ac = get_distance(a,c)
    alpha = cosinelaw(ab,ac,bc)
    beta = cosinelaw(ab,bc,ac)
    gamma = cosinelaw(ac,bc,ab)
    return alpha,beta,gamma


def add_triangle_gadget(mv, strip_width, t, strip=[]):
    a, b, c = t
    alpha,beta,gamma = calculate_angles(a,b,c)
    height = get_distance(a,b) * math.sin(beta)
    alternater = itertools.cycle([Direction.R, Direction.L])
    y=0
    while(y < height):
        direction = next(alternater)
        add_plain((get_distance(b,c)*y)/height + corner_extension(gamma,beta,direction,strip_width), strip)
        add_corner_gadget(mv,direction, strip)
        y+=strip_width

In [19]:
stripwidth = 10
strip = []
add_triangle_gadget(MV.MOUNTAIN,stripwidth,[[50,-20,200],[50,-20,0],[50,180,0]], strip) 
render(strip, stripwidth)

In [20]:
import igl
from meshplot import plot

# ligigl naming convention ahead:
# v: array of vertex coordinates
# f: array of faces (defined by the vertex ids in clockwise order)
v, f = igl.read_triangle_mesh("cube.stl")

stripwidth = 4
strip = []
for i, face in enumerate(f):
    mountain_valley_flag = MV.MOUNTAIN if (i%2 == 0) else MV.VALLEY
    face_coordinate_list = [v[vertex_id] for vertex_id in face]
    add_triangle_gadget(mountain_valley_flag, stripwidth, face_coordinate_list, strip)

render(strip, stripwidth)